In [3]:
# pip install scikit-learn

In [4]:
# from sklearn import datasets
# from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
# import os
import json
import zipfile
import numpy as np


In [5]:
with open("data/output.json", 'r') as f:
    data = json.load(f)

In [6]:
#faking data
liked_movies = data["liked_movies"]
disliked_movies = ["ill_see_you_in_my_dreams", "sometime_other_than_now", "1001294-asylum", "good_boys_2019"]
#disliked_movies = data["disliked_movies"]
train_id = liked_movies + disliked_movies

In [7]:
# train_i

['back_to_the_future',
 '1006364-dune',
 'maze_2019',
 'alien',
 'the_amazing_spider_man',
 'spiderman_2',
 'iron_man',
 'heat_1995',
 'ill_see_you_in_my_dreams',
 'sometime_other_than_now',
 '1001294-asylum',
 'good_boys_2019']

In [8]:
# Read your CSV file
og_data = pd.read_csv('final_data.csv').reset_index()
# titles = og_data['id']
# mod_data = pd.get_dummies(og_data.drop(columns = "id"))
# print(mod_data.columns)

In [9]:
og_data["liked"] = og_data["id"].isin(liked_movies).astype(int)
# print(og_data.head())

   index                id  runtimeMinutes          originalLanguage  \
0      0      alone_2020_2              98                   English   
1      1         maze_2019              92  English (United Kingdom)   
2      2    good_boys_2019              95                   English   
3      3  us_and_them_2018             120                   Chinese   
4      4    1001294-asylum              88                   English   

   tomatoMeter  audienceScore  liked  p_drama  p_mystery  p_thriller  ...  \
0           91             55      0        1          1           1  ...   
1           92             46      1        1          0           0  ...   
2           80             86      0        0          0           0  ...   
3          100             90      0        1          0           0  ...   
4           67             50      0        0          0           0  ...   

   p_clive donner  p_richard gray  p_philip gelatt  p_justin folk  \
0               0               0  

In [10]:
index_id_tbl = og_data[["index", "id"]]
# print(index_id_tbl.head())

   index                id
0      0      alone_2020_2
1      1         maze_2019
2      2    good_boys_2019
3      3  us_and_them_2018
4      4    1001294-asylum


In [11]:
mod_tbl = pd.get_dummies(og_data.drop(columns = "id"))
# mod_tbl

In [12]:
train_id_idx = index_id_tbl[index_id_tbl["id"].isin(train_id)]["index"]
# train_id_idx

In [13]:
train_data = mod_tbl[mod_tbl["index"].isin(train_id_idx)]
# train_data["liked"]

In [14]:
test_data = mod_tbl[~mod_tbl["index"].isin(train_id_idx)]
# test_data

In [15]:
# Select training data (drop the 'Title' column)
# X = mod_data.drop(columns = ["liked"])
# y = og_data[["id","liked"]]

In [16]:
# Suppose train_data and test_data are already split

# Apply one-hot encoding
# train_data = pd.get_dummies(train_data)
# test_data = pd.get_dummies(test_data)

# # Align the columns between train and test (VERY IMPORTANT)
# train_data, test_data = train_data.align(test_data, join='left', axis=1, fill_value=0)


In [17]:


# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=446)

# # Scale features (important for SVMs!)
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)


In [18]:
X_train = pd.get_dummies(train_data.drop(columns = "liked"))
X_test = pd.get_dummies(test_data.drop(columns = "liked"))
y_train = train_data["liked"]

In [19]:
# X_train

In [20]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [21]:

svm_model = SVC(kernel='linear')
svm_model.fit(X_train_scaled, y_train)

SVC(kernel='linear')

In [22]:
predictions = svm_model.predict(X_test_scaled)
# print(len(predictions))

In [23]:
pred_tbl =  pd.DataFrame({"idx" : X_test["index"],
                          "preds": predictions,
                          
                          })
pred_tbl["id"] = og_data[og_data["index"].isin(X_test["index"])]["id"]
pred_tbl

,idx,preds,id
0,0,1,alone_2020_2
3,3,1,us_and_them_2018
6,6,0,exorcism_of_emily_rose
7,7,0,jungle_fever
8,8,0,the_masque_of_the_red_death_1964
...,...,...,...
5869,5869,0,sordid_lives
5870,5870,1,raising_bertie_2017
5871,5871,0,everyday_2013
5872,5872,1,the-carey-treatment-a-case-of-murder-emergency...


In [86]:
# pred_tbl.to_json("data/test_preds.json", orient="records", indent=4)

In [22]:

# from sklearn.metrics import accuracy_score
# print("Accuracy:", accuracy_score(y_test[1:100], predictions))

Accuracy: 0.9090909090909091


In [24]:
decision_values = svm_model.decision_function(X_test_scaled)
decision_values

array([ 0.04223363,  0.18658005, -0.03648727, ..., -0.02230499,
        0.05319534,  0.04236444], shape=(5863,))

In [29]:
# ranked_indices = np.argsort(decision_values)[::-1]  # Reverse order


In [ ]:
# top_10 = liked_ranking.nlargest(10, "mod_confidence")
# top_10

,idx,preds,id,mod_confidence
2324,2324,1,best_of_youth,1.067686
4536,4536,1,norte_the_end_of_history,0.919354
260,260,1,ludwig,0.853848
5839,5839,1,1048378-edvard_munch,0.846491
2779,2779,1,rrr,0.834725
2035,2035,1,jhund,0.832022
3273,3273,1,khakee,0.812493
906,906,1,lawrence_of_arabia,0.805021
2736,2736,1,iron_man_3,0.799624
223,223,1,bandit_queen,0.792784


# Cosine Similarity

In [ ]:
# X_train

In [39]:
liked_movies = train_data[train_data["liked"] == 1]  
cos_sim_scores = cosine_similarity(test_data.drop(columns="liked"), liked_movies.drop(columns="liked")).mean(axis=1) 

In [40]:
# cos_sim_scores[1:100]

array([0.54936199, 0.55715028, 0.56717986, 0.56998559, 0.54040763,
       0.57009408, 0.57575731, 0.60299496, 0.60394476, 0.60005719,
       0.60206682, 0.61923697, 0.59952049, 0.60475072, 0.61970939,
       0.62139863, 0.62321952, 0.63660403, 0.61677191, 0.62024926,
       0.61783064, 0.64551598, 0.64795408, 0.63863486, 0.65565521,
       0.64704645, 0.65886873, 0.66034263, 0.67055787, 0.69173823,
       0.66022568, 0.69537001, 0.66460573, 0.70628304, 0.70540366,
       0.71977973, 0.68339625, 0.67485048, 0.69457189, 0.68319881,
       0.69211085, 0.72010751, 0.70719303, 0.71547501, 0.71030572,
       0.72720586, 0.71826182, 0.70460836, 0.72847817, 0.71845721,
       0.74563975, 0.72447077, 0.75210434, 0.71817938, 0.76044936,
       0.77062367, 0.74430524, 0.77856599, 0.78696654, 0.70053649,
       0.76331236, 0.74965035, 0.74524328, 0.74207255, 0.80895386,
       0.75932304, 0.76901815, 0.73030909, 0.80554595, 0.75186302,
       0.76726775, 0.76232947, 0.7569137 , 0.81510197, 0.77414

In [43]:
final_ranking = decision_values * cos_sim_scores
final_ranking

array([ 0.0225761 ,  0.10249999, -0.02032889, ..., -0.01503199,
        0.03577289,  0.02849142], shape=(5863,))

In [45]:
pred_tbl

,idx,preds,id,mod_confidence,scores
0,0,1,alone_2020_2,0.042234,0.022576
3,3,1,us_and_them_2018,0.186580,0.102500
6,6,0,exorcism_of_emily_rose,-0.036487,-0.020329
7,7,0,jungle_fever,-0.019025,-0.010791
8,8,0,the_masque_of_the_red_death_1964,-0.006545,-0.003730
...,...,...,...,...,...
5869,5869,0,sordid_lives,-0.218426,-0.147290
5870,5870,1,raising_bertie_2017,0.163915,0.110772
5871,5871,0,everyday_2013,-0.022305,-0.015032
5872,5872,1,the-carey-treatment-a-case-of-murder-emergency...,0.053195,0.035773


In [44]:
pred_tbl["scores"] = final_ranking
liked_ranking = pred_tbl[pred_tbl["preds"] == 1]
liked_ranking

,idx,preds,id,mod_confidence,scores
0,0,1,alone_2020_2,0.042234,0.022576
3,3,1,us_and_them_2018,0.186580,0.102500
9,9,1,host_2020,0.215524,0.116471
13,13,1,adventure_of_sherlock_holmes_smarter_brother,0.028658,0.017308
15,15,1,the_laureate,0.327981,0.197467
...,...,...,...,...,...
5867,5867,1,the_mill_and_the_cross,0.014777,0.009963
5868,5868,1,total_dhamaal,0.221374,0.149077
5870,5870,1,raising_bertie_2017,0.163915,0.110772
5872,5872,1,the-carey-treatment-a-case-of-murder-emergency...,0.053195,0.035773


In [46]:
top_10 = liked_ranking.nlargest(10, "scores")
top_10

,idx,preds,id,mod_confidence,scores
2324,2324,1,best_of_youth,1.067686,0.784555
260,260,1,ludwig,0.853848,0.692472
223,223,1,bandit_queen,0.792784,0.676731
4536,4536,1,norte_the_end_of_history,0.919354,0.636527
906,906,1,lawrence_of_arabia,0.805021,0.628305
515,515,1,terminator_2_judgment_day,0.773496,0.624816
2035,2035,1,jhund,0.832022,0.600140
2779,2779,1,rrr,0.834725,0.587466
5839,5839,1,1048378-edvard_munch,0.846491,0.579158
795,795,1,john_wick_chapter_4,0.739584,0.576974


In [48]:
top_10.to_json("data/test_preds.json", orient="records", indent=4)